## there's always stuff that can be done

    1. create aliases for opening your different notes packages
    2. grep command.  a bash command for searching your notes folder for the code snippet
    3. create a repo for this project
    4. 

In [112]:
import sys
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
from urllib.request import urlopen
import string
import re
import unicodedata
import datetime as dt


In [2]:
'''
Get NBA team names and put them in a list format
'''
nba_tables = pd.read_html('https://en.wikipedia.org/wiki/National_Basketball_Association#Teams', index_col=0, header=0)
teams = nba_tables[3]
teams.reset_index(inplace=True)
teams = teams[teams.Team != 'Western Conference']
teams = teams.iloc[1:, 1:-1]
teams_list = list(teams.Team)
clean_teams = [team.replace(' ', '_') for team in teams_list]

In [15]:
nba_players = []

In [16]:
'''
Get roster table from each team's wiki page
'''

%%time 
for team in clean_teams:
	url = f'https://en.wikipedia.org/wiki/{team}'
	html = urlopen(url)
	soup = BeautifulSoup(html, 'lxml')
	tables = soup.find_all('table', {"class": "toccolours"})
	table = tables[0]
	tbody = table.find_all('tbody')[0]
	actualtable = tbody.find_all('table')[0]
	actualrows = actualtable.find_all('tr')[1:]
	for tr in actualrows:
		td = tr.find_all('td')
		row = [unicodedata.normalize('NFKD', i.text.strip()) for i in td]   # gets rid of the weird \x0 spaces
		row.append(team.replace('_', ' '))          # add column for team.  may just want to use 2nd index later 
		nba_players.append(row)

CPU times: user 23.1 s, sys: 221 ms, total: 23.3 s
Wall time: 31.7 s


In [14]:
headers = ['position', 'number', 'name', 'height', 'weight', 'dob', 'college', 'team']

In [17]:
nba_df = pd.DataFrame(data=nba_players, columns=headers)

In [42]:
nba_df.dob = nba_df.dob.str.replace('–', '-')       # need to find a way around those long hyphens besides manually
nba_df.dob = pd.to_datetime(nba_df.dob)

In [75]:
nba_df['height_in'] = nba_df['height'].apply(lambda x: x.split('(')[0])
nba_df.height_in = nba_df['height'].apply(lambda x: (int(x.split(' ')[0]) * 12) + (int(x.split(' ')[2])))
nba_df['weight_int'] = nba_df['weight'].apply(lambda x: int(x.split(' ')[0]))
nba_df['bmi'] = (703 * nba_df.weight_int) / (nba_df.height_in**2)

In [245]:
'''
YOUNGEST TEAM IN NBA - METHOD #1: 
converts date of birth to nanoseconds in order to run the mean, then converts back to dt
'''
nba_df['dob_ns'] = nba_df.dob.values.astype(np.int64)
pd.to_datetime(nba_df.groupby(by='team')['dob_ns'].mean()).sort_values(ascending=False)

team
Minnesota Timberwolves   1996-02-04 13:30:00.000000000
Atlanta Hawks            1995-11-03 16:00:00.000000000
Charlotte Hornets        1995-07-21 07:30:00.000000000
Memphis Grizzlies        1995-07-03 05:38:49.411764736
Phoenix Suns             1995-06-25 09:00:00.000000000
New York Knicks          1995-06-03 22:35:17.647058816
Chicago Bulls            1995-04-11 09:52:56.470588288
Boston Celtics           1994-12-30 00:00:00.000000000
Cleveland Cavaliers      1994-11-22 18:21:10.588235264
Washington Wizards       1994-11-19 00:00:00.000000000
Oklahoma City Thunder    1994-10-29 12:42:21.176470528
Golden State Warriors    1994-10-12 12:55:23.076923136
Detroit Pistons          1994-10-11 11:17:38.823529472
Portland Trail Blazers   1994-09-27 09:52:56.470588288
Indiana Pacers           1994-08-21 08:28:14.117647104
Denver Nuggets           1994-07-18 14:07:03.529411712
New Orleans Pelicans     1994-06-27 18:40:00.000000000
Utah Jazz                1994-01-11 08:28:14.117647104
Sacra

In [ ]:
'''
YOUNGEST TEAM IN NBA - METHOD #2: 
subtracts each player's DoB from right now which creates a timedelta object, then calculate a year figure in standard int format
'''
nba_df['current_age'] = nba_df.dob.apply(lambda x: (dt.datetime.now() - x).days / 365.25)

In [262]:
# Top 5 youngest teams
nba_df.groupby('team')['current_age'].mean().sort_values(ascending=True).head()

team
Minnesota Timberwolves    24.565195
Atlanta Hawks             24.819530
Charlotte Hornets         25.107974
Memphis Grizzlies         25.157467
Phoenix Suns              25.178987
Name: current_age, dtype: float64

In [263]:
# Top 5 oldest teams
nba_df.groupby('team')['current_age'].mean().sort_values(ascending=False).head()

team
Houston Rockets         30.635262
Milwaukee Bucks         29.813746
Los Angeles Lakers      29.485132
Los Angeles Clippers    28.353183
Dallas Mavericks        27.916648
Name: current_age, dtype: float64

In [255]:
# Top 5 heaviest teams
nba_df.groupby('team')['weight_int'].mean().sort_values(ascending=False).head()

team
Denver Nuggets            225.588235
Milwaukee Bucks           225.235294
Boston Celtics            224.823529
Portland Trail Blazers    223.882353
Cleveland Cavaliers       222.235294
Name: weight_int, dtype: float64

In [256]:
# Top 5 tallest teams
nba_df.groupby('team')['height_in'].mean().sort_values(ascending=False).head()

team
Portland Trail Blazers    79.588235
Denver Nuggets            79.529412
Chicago Bulls             79.176471
San Antonio Spurs         79.000000
Memphis Grizzlies         78.882353
Name: height_in, dtype: float64

In [260]:
# Top 10 colleges by representation
nba_df.pivot_table(index='college', aggfunc='size').sort_values(ascending=False).head(10)

college
Kentucky          28
Duke              24
North Carolina    14
UCLA              13
Texas             11
Kansas            11
France            10
Michigan          10
Indiana            9
Arizona            9
dtype: int64